### Data Cleaning & Merging
##### Author : Shubhajit Basak
##### Merge data from multi source Open Data

In [1]:
# Import Packages
import pandas as pd
import numpy as np 
from geopy.geocoders import Nominatim
from enum import Enum
import xml.etree.ElementTree as ET
from datetime import datetime
import folium

#### Generic Methods that will used later

In [2]:
# Method To trim extra spaces accross the data of a dataframe
def trimAllColumns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trimStrings = lambda x: x.strip() if type(x) is str else x
    return df.applymap(trimStrings)


#Method to get the Location from Latitude and Longitude Data
geolocator = Nominatim(user_agent="my_app", country_bias='Ireland', timeout=None)

def getLocatonFromLatLong(lat,long):
    location = geolocator.reverse([(lat,long)])
    return location.address.split(", ")[-6]

#### Load the three datasets

In [3]:
#Read and load the DCC and DLR data from csv
dcc = pd.read_csv("dcc_pitches.csv")
dlr = pd.read_csv("dlr_pitches.csv")

In [4]:
# Load the data for FCC from a XML 
# Code Reference 
# https://www.kaggle.com/ysviru/simple-analysis-of-healthcare-job-postings

#Build the XML Model 
class XMLTagsUpperLevel:
    """
    This class defines the XML tag constants at the higher level of XML tree. 
    """
    TABLE = "Playing_Pitches-table"
    PITCHES = "Playing_Pitches"

class XMLTagsLowerLevel(Enum):
    """
    This class defines all the XML tag constants that are one level below the <Playing_Pitches> tag. This is defined as an
    enumerated type for ease of iterating over all tags.
    """
    FACILITY_TYPE = "FACILITY_TYPE"
    FACILITY_NAME = "FACILITY_NAME"
    LOCATION = "LOCATION"
    LAT = "LAT"
    LONG = "LONG"

#Class XML Parser
class XMLParser:

    def __init__(self, file_path="fcc_pitches.xml"):
        """
        Initializes the XMLParser class instance.
        :param file_path: Path to input xml file containing the data.
        """
        self.file_path = file_path


    def xml_to_pandas_df(self):
        """
        Using the standard xml python library, we parse the data xml file and convert the xml data to a pandas
        data frame.
        :return: A pandas data frame instance containing all the data.
        """
        tree = ET.parse(self.file_path)
        root = tree.getroot()

        pitches_data = dict()
        for tag in XMLTagsLowerLevel:
            pitches_data[tag.value] = []

        for i, page in enumerate(root.findall(XMLTagsUpperLevel.TABLE)):
            for j, record in enumerate(page.findall(XMLTagsUpperLevel.PITCHES)):
                for tag in XMLTagsLowerLevel:
                    temp = record.find(tag.value)
                    if temp is not None:
                        pitches_data[tag.value].append(temp.text)
                    else:
                        pitches_data[tag.value].append("")

        return pd.DataFrame(data=pitches_data)


In [5]:
# Read and load the FCC Data 
file_path = "fcc_pitches.xml"
parser = XMLParser(file_path)
fcc = parser.xml_to_pandas_df()

#### Check few records and the datacount to get the count of missing values of different field for all the files

In [6]:
## DCC Data
dcc.head()

,PARK,AREA,CLUBNAME,LEAGUE,Unnamed: 4
0,ALBERT COLLEGE,NORTH WEST,DRUMCONDRA F.C (Snr),AMATEUR FOOTBALL LEAGUE,NaN
1,ALBERT COLLEGE,NORTH WEST,GLASNEVIN AFC,AMATEUR FOOTBALL LEAGUE,NaN
2,BEECHILL,SOUTH EAST,BALLSBRIDGE FC,AMATEUR FOOTBALL LEAGUE,NaN
3,BELCAMP,NORTH CENTRAL,NEWTOWN CELTIC,AMATEUR FOOTBALL LEAGUE,NaN
4,BELCAMP,NORTH CENTRAL,VIANNEY BOYS,AMATEUR FOOTBALL LEAGUE,NaN


In [7]:
# dcc Count 
print(dcc.count())
# counts of null values in different columns
print(dcc.isnull().sum(axis = 0))

PARK          250
AREA          250
CLUBNAME      250
LEAGUE        250
Unnamed: 4      0
dtype: int64
PARK            0
AREA            0
CLUBNAME        0
LEAGUE          0
Unnamed: 4    250
dtype: int64


<b> Observation </b> - There are total 250 records for DCC and none of the column has missing values, though the Latitude, Longitude, Location and Pitch Type are missing

In [8]:
dlr.head()

,Location,Number,Size,Latitude,Longitude
0,Kilbogget Park,1,Snr,53.257242,-6.140665
1,NaN,2,SSG,53.257614,-6.139882
2,NaN,3,SSG,53.257842,-6.139265
3,NaN,4,SSG,53.257098,-6.139094
4,NaN,5,SGG,53.256674,-6.140134


In [9]:
# dlr Count 
print(dlr.count())
# counts of null values in different columns
print(dlr.isnull().sum(axis = 0))

Location     15
Number       60
Size         58
Latitude     60
Longitude    60
dtype: int64
Location     46
Number        1
Size          3
Latitude      1
Longitude     1
dtype: int64


<b> Observation </b> - There are total 61 records for DLR and in which for 46 records Loaction is missing, Pitch Number, Latitude and Longitude are missing for one record and Pitch Size is missing for 3 values

In [10]:
fcc.head()

,FACILITY_NAME,FACILITY_TYPE,LAT,LOCATION,LONG
0,Balbriggan Town Park,All weather pitches,53.6049596246817,Balbriggan,-6.18235291959051
1,Balheary Reservoir,All weather pitches,53.4727096370551,Swords,-6.22301521551813
2,Town Park,All weather pitches,53.5771135903791,Skerries,-6.11107205744599
3,St. Mologa's Park,All weather pitches,53.6176672458903,Balbriggan,-6.18936794084573
4,Seagrange Park,Basketball Court,53.3966674985382,None,-6.13535180348378


In [11]:
# fcc Count 
print(fcc.count())
# counts of null values in different columns
print(fcc.isnull().sum(axis = 0))

FACILITY_NAME    96
FACILITY_TYPE    96
LAT              96
LOCATION         73
LONG             96
dtype: int64
FACILITY_NAME     0
FACILITY_TYPE     0
LAT               0
LOCATION         23
LONG              0
dtype: int64


<b> Observation </b> - There are total 96 records for FCC and in which for 23 records Loaction is missing

#### Data Quality Enhancement

In [12]:
#Trim all the string columns of the three dataset
dcc = trimAllColumns(dcc)
dlr = trimAllColumns(dlr)
fcc = trimAllColumns(fcc)

In [13]:
## Clean the DCC Dataset 

#Drop the extra column at right (which came due to an extra comma) from DCC Dataset
dcc = dcc.drop(dcc.columns[4],1)

# Rename the column names of DCC 
dcc.columns = ['Park','Area','ClubName','League']

dcc.head()

,Park,Area,ClubName,League
0,ALBERT COLLEGE,NORTH WEST,DRUMCONDRA F.C (Snr),AMATEUR FOOTBALL LEAGUE
1,ALBERT COLLEGE,NORTH WEST,GLASNEVIN AFC,AMATEUR FOOTBALL LEAGUE
2,BEECHILL,SOUTH EAST,BALLSBRIDGE FC,AMATEUR FOOTBALL LEAGUE
3,BELCAMP,NORTH CENTRAL,NEWTOWN CELTIC,AMATEUR FOOTBALL LEAGUE
4,BELCAMP,NORTH CENTRAL,VIANNEY BOYS,AMATEUR FOOTBALL LEAGUE


In [14]:
## Clean and modify FCC Dataset 

#Reorder the columns
fcc = fcc[["FACILITY_NAME","FACILITY_TYPE","LOCATION","LAT","LONG"]]

#Rename the FCC Columns so that it will help while merging the data
fcc.columns = ['Park','TypeOfPitch','Location','Latitude','Longitude']
fcc.head()

,Park,TypeOfPitch,Location,Latitude,Longitude
0,Balbriggan Town Park,All weather pitches,Balbriggan,53.6049596246817,-6.18235291959051
1,Balheary Reservoir,All weather pitches,Swords,53.4727096370551,-6.22301521551813
2,Town Park,All weather pitches,Skerries,53.5771135903791,-6.11107205744599
3,St. Mologa's Park,All weather pitches,Balbriggan,53.6176672458903,-6.18936794084573
4,Seagrange Park,Basketball Court,None,53.3966674985382,-6.13535180348378


#### Check for duplicate and drop if any in all the 3 datasets

In [15]:
#dlr
print(len(dlr))
dlr = dlr.drop_duplicates(subset=list(dlr), keep=False)
print(len(dlr))

#dcc
print(len(dcc))
dcc = dcc.drop_duplicates(subset=list(dcc), keep=False)
print(len(dcc))

#fcc
print(len(fcc))
fcc = fcc.drop_duplicates(subset=list(fcc), keep=False)
print(len(fcc))


61
61
250
250
96
96


We have found there is no such duplicate data in terms of all column present in the individual dataset

#### Rename the column names of the dataframes to get a unified model

In [16]:
print(list(dcc))
print(list(dlr))
print(list(fcc))

['Park', 'Area', 'ClubName', 'League']
['Location', 'Number', 'Size', 'Latitude', 'Longitude']
['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude']


In [17]:
#Rename the DLR Columns so that it will help while merging the data
dlr.columns = ['Park','TypeOfPitch','Size','Latitude','Longitude']
dlr.head()

,Park,TypeOfPitch,Size,Latitude,Longitude
0,Kilbogget Park,1,Snr,53.257242,-6.140665
1,NaN,2,SSG,53.257614,-6.139882
2,NaN,3,SSG,53.257842,-6.139265
3,NaN,4,SSG,53.257098,-6.139094
4,NaN,5,SGG,53.256674,-6.140134


In [18]:
print(list(dcc))
print(list(dlr))
print(list(fcc))

['Park', 'Area', 'ClubName', 'League']
['Park', 'TypeOfPitch', 'Size', 'Latitude', 'Longitude']
['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude']


#### Modify the DLR Data

In [19]:
## Clean and fillthe missing values in FCC Dataset 

# As the data in the first column for DLR is missing, fill it with the previous one
dlr['Park'] = dlr['Park'].fillna(method='ffill')
dlr.head()

,Park,TypeOfPitch,Size,Latitude,Longitude
0,Kilbogget Park,1,Snr,53.257242,-6.140665
1,Kilbogget Park,2,SSG,53.257614,-6.139882
2,Kilbogget Park,3,SSG,53.257842,-6.139265
3,Kilbogget Park,4,SSG,53.257098,-6.139094
4,Kilbogget Park,5,SGG,53.256674,-6.140134


In [20]:
# Add the Column Location in DLR Dataset
dlr = dlr.assign(Location= np.nan)

In [21]:
list(dlr)

['Park', 'TypeOfPitch', 'Size', 'Latitude', 'Longitude', 'Location']

In [22]:
# Reorder the columns for dlr
dlr = dlr[['Park', 'TypeOfPitch','Location', 'Size', 'Latitude', 'Longitude']]

In [23]:
#add Column City Council
dlr = dlr.assign(CityCouncil = 'Dún Laoghaire–Rathdown')

#Drop Pitch Size 
dlr = dlr.drop(dlr.columns[3],1)

In [24]:
list(dlr)

['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude', 'CityCouncil']

In [25]:
# Update the locatiion from longitude and latitude if they present
for i,row in dlr.iterrows():
    if not np.isnan(dlr.iloc[i,4]):
        dlr.iloc[i,2] = getLocatonFromLatLong(dlr.iloc[i,3],dlr.iloc[i,4])

In [26]:
dlr.head()

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil
0,Kilbogget Park,1,Killiney,53.257242,-6.140665,Dún Laoghaire–Rathdown
1,Kilbogget Park,2,Killiney,53.257614,-6.139882,Dún Laoghaire–Rathdown
2,Kilbogget Park,3,Killiney,53.257842,-6.139265,Dún Laoghaire–Rathdown
3,Kilbogget Park,4,Killiney,53.257098,-6.139094,Dún Laoghaire–Rathdown
4,Kilbogget Park,5,Killiney,53.256674,-6.140134,Dún Laoghaire–Rathdown


In [27]:
dlr.count()

Park           61
TypeOfPitch    60
Location       60
Latitude       60
Longitude      60
CityCouncil    61
dtype: int64

In [28]:
#as wehave found that latitude longitude and location is missing for a row lets identify the data 
dlr[dlr.isnull().any(axis=1)]

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil
28,Shanganagh Castle,Temporary pitch,NaN,NaN,NaN,Dún Laoghaire–Rathdown
60,Mount Albany,NaN,Blackrock,53.286265,-6.173321,Dún Laoghaire–Rathdown


##### As there are only two records with missing values we will manually update in two steps 

In [29]:
# Update the location, latitude and longitude of the missing value
location1 = geolocator.geocode("Shanganagh Castle")
dlr.iloc[28,3] = location1.latitude
dlr.iloc[28,4] = location1.longitude
dlr.iloc[28,2] = location1.address.split(", ")[-6]
print(dlr.iloc[28,:])

Park                Shanganagh Castle
TypeOfPitch           Temporary pitch
Location                    Dublin 18
Latitude                      53.2233
Longitude                     -6.1237
CityCouncil    Dún Laoghaire–Rathdown
Name: 28, dtype: object


In [30]:
# Update the pitch type with the most common pitchtype in the dataset
# as there is only one value is missing we can make this assumption
dlr.iloc[60,1] =  dlr["TypeOfPitch"].value_counts().argmax()

In [31]:
#verify if there is any further null value 
dlr[dlr.isnull().any(axis=1)]

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil


In [32]:
dlr.head()

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil
0,Kilbogget Park,1,Killiney,53.257242,-6.140665,Dún Laoghaire–Rathdown
1,Kilbogget Park,2,Killiney,53.257614,-6.139882,Dún Laoghaire–Rathdown
2,Kilbogget Park,3,Killiney,53.257842,-6.139265,Dún Laoghaire–Rathdown
3,Kilbogget Park,4,Killiney,53.257098,-6.139094,Dún Laoghaire–Rathdown
4,Kilbogget Park,5,Killiney,53.256674,-6.140134,Dún Laoghaire–Rathdown


#### Modify the FCC Dataset

In [33]:
## Check for Null Values 
fcc.count()

Park           96
TypeOfPitch    96
Location       73
Latitude       96
Longitude      96
dtype: int64

In [34]:
list(fcc)

['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude']

Only Location data is missing for this dataset

In [35]:
#Fill the locations where not present
for i,row in fcc.iterrows():
    if fcc.iloc[i,2] is None:
        fcc.iloc[i,2] = getLocatonFromLatLong(fcc.iloc[i,3],fcc.iloc[i,4])

In [36]:
# Add new column as City Council
fcc = fcc.assign(CityCouncil = 'Fingal')

In [37]:
#verify if there is any further null value 
fcc[fcc.isnull().any(axis=1)]

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil


In [38]:
fcc.head()

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil
0,Balbriggan Town Park,All weather pitches,Balbriggan,53.6049596246817,-6.18235291959051,Fingal
1,Balheary Reservoir,All weather pitches,Swords,53.4727096370551,-6.22301521551813,Fingal
2,Town Park,All weather pitches,Skerries,53.5771135903791,-6.11107205744599,Fingal
3,St. Mologa's Park,All weather pitches,Balbriggan,53.6176672458903,-6.18936794084573,Fingal
4,Seagrange Park,Basketball Court,Dublin 13,53.3966674985382,-6.13535180348378,Fingal


#### Modifying DCC Dataset

In [39]:
# Add Lititude, Longitude, Location, Type Of Pitch and City Council in DCC Dataset
dcc = dcc.assign(Latitude = np.nan, Longitude = np.nan, Location = None,CityCouncil = "Dublin City Council", 
                 TypeOfPitch = "Not Available")
dcc.head()

,Park,Area,ClubName,League,CityCouncil,Latitude,Location,Longitude,TypeOfPitch
0,ALBERT COLLEGE,NORTH WEST,DRUMCONDRA F.C (Snr),AMATEUR FOOTBALL LEAGUE,Dublin City Council,NaN,None,NaN,Not Available
1,ALBERT COLLEGE,NORTH WEST,GLASNEVIN AFC,AMATEUR FOOTBALL LEAGUE,Dublin City Council,NaN,None,NaN,Not Available
2,BEECHILL,SOUTH EAST,BALLSBRIDGE FC,AMATEUR FOOTBALL LEAGUE,Dublin City Council,NaN,None,NaN,Not Available
3,BELCAMP,NORTH CENTRAL,NEWTOWN CELTIC,AMATEUR FOOTBALL LEAGUE,Dublin City Council,NaN,None,NaN,Not Available
4,BELCAMP,NORTH CENTRAL,VIANNEY BOYS,AMATEUR FOOTBALL LEAGUE,Dublin City Council,NaN,None,NaN,Not Available


In [40]:
#Change the order of the columns so thet it matches with data model
dcc = dcc[['Park','TypeOfPitch', 'Location', 'Latitude', 'Longitude','CityCouncil', 'Area', 'ClubName', 'League' ]]
list(dcc)

['Park',
 'TypeOfPitch',
 'Location',
 'Latitude',
 'Longitude',
 'CityCouncil',
 'Area',
 'ClubName',
 'League']

##### Add the Location Latitude and Longitude Data to DCC

In [41]:
# Search the Location by either Club Name or the Park Name and update the latitude longitude and loaction information

startTime = datetime.now()
for i, row in dcc.iterrows():
    locationOption1 = geolocator.geocode(dcc.iloc[i,7]) # Search by Club Name
    locationOption2 = geolocator.geocode(dcc.iloc[i,0]+" PARK DUBLIN") # Search by Park Name
    locationOption3 = geolocator.geocode(max(dcc.iloc[i,7].split(),key = len)+" DUBLIN") # Search by the highest length word of Club Name
    if((locationOption1 is not None) and "County Dublin" in locationOption1.address ):
        dcc.iloc[i,3] = locationOption1.latitude
        dcc.iloc[i,4] = locationOption1.longitude
        dcc.iloc[i,2] = locationOption1.address.split(", ")[-6]
    elif((locationOption2 is not None) and "County Dublin" in locationOption2.address):
        dcc.iloc[i,3] = locationOption2.latitude
        dcc.iloc[i,4] = locationOption2.longitude
        dcc.iloc[i,2] = locationOption2.address.split(", ")[-6]
    elif((locationOption3 is not None) and "County Dublin" in locationOption3.address):
        dcc.iloc[i,3] = locationOption3.latitude
        dcc.iloc[i,4] = locationOption3.longitude
        dcc.iloc[i,2] = locationOption3.address.split(", ")[-6]
    else:
        print("Data Not Found for:")
        print(dcc.iloc[i,0]+" DUBLIN")
        print(dcc.iloc[i,7])
endTime = datetime.now()
print("Total Time takes to fetch all data :")
print(endTime - startTime)

Data Not Found for:
JOHN PAUL DUBLIN
BEGGSBORO AFC SCHOOLBOYS
Data Not Found for:
FR. COLLINS DUBLIN
SCHALKE ROVERS
Total Time takes to fetch all data :
0:04:05.940783


In [42]:
#Check DCC Data for any missing values
dcc[dcc.isnull().any(axis=1)]

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil,Area,ClubName,League
82,JOHN PAUL,Not Available,None,NaN,NaN,Dublin City Council,CENTRAL,BEGGSBORO AFC SCHOOLBOYS,DUBLIN DISTRICT SCHOOLBOYS LEAGUE
227,FR. COLLINS,Not Available,None,NaN,NaN,Dublin City Council,NORTH CENTRAL,SCHALKE ROVERS,UNITED CHURCHES FOOTBALL LEAGUE


##### As for two records the data is not available we will manually search for similar records in the dataset and update the two records 

In [43]:
# Search for data with Club Name BEGGSBORO
dcc[dcc["ClubName"].str.contains("BEGGSBORO")]

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil,Area,ClubName,League
82,JOHN PAUL,Not Available,None,NaN,NaN,Dublin City Council,CENTRAL,BEGGSBORO AFC SCHOOLBOYS,DUBLIN DISTRICT SCHOOLBOYS LEAGUE
83,JOHN PAUL,Not Available,Dublin 7,53.36496,-6.287479,Dublin City Council,CENTRAL,BEGGSBORO A.F.C,DUBLIN DISTRICT SCHOOLBOYS LEAGUE


In [44]:
# Copy from the available Column
dcc.iloc[82,2] = dcc.iloc[83,2]
dcc.iloc[82,3] = dcc.iloc[83,3]
dcc.iloc[82,4] = dcc.iloc[83,4]
dcc[dcc["ClubName"].str.contains("BEGGSBORO")]

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil,Area,ClubName,League
82,JOHN PAUL,Not Available,Dublin 7,53.36496,-6.287479,Dublin City Council,CENTRAL,BEGGSBORO AFC SCHOOLBOYS,DUBLIN DISTRICT SCHOOLBOYS LEAGUE
83,JOHN PAUL,Not Available,Dublin 7,53.36496,-6.287479,Dublin City Council,CENTRAL,BEGGSBORO A.F.C,DUBLIN DISTRICT SCHOOLBOYS LEAGUE


In [45]:
#Search for data with clubname as  SCHALKE
dcc[dcc["ClubName"].str.contains("SCHALKE")]

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil,Area,ClubName,League
227,FR. COLLINS,Not Available,None,NaN,NaN,Dublin City Council,NORTH CENTRAL,SCHALKE ROVERS,UNITED CHURCHES FOOTBALL LEAGUE
232,ST ANNES,Not Available,Shankill-Shanganagh ED,53.228847,-6.11739,Dublin City Council,NORTH CENTRAL,SCHALKE ROVERS,UNITED CHURCHES FOOTBALL LEAGUE


In [46]:
# Copy from the available Column
dcc.iloc[227,2] = dcc.iloc[232,2]
dcc.iloc[227,3] = dcc.iloc[232,3]
dcc.iloc[227,4] = dcc.iloc[232,4]
dcc[dcc["ClubName"].str.contains("SCHALKE")]

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil,Area,ClubName,League
227,FR. COLLINS,Not Available,Shankill-Shanganagh ED,53.228847,-6.11739,Dublin City Council,NORTH CENTRAL,SCHALKE ROVERS,UNITED CHURCHES FOOTBALL LEAGUE
232,ST ANNES,Not Available,Shankill-Shanganagh ED,53.228847,-6.11739,Dublin City Council,NORTH CENTRAL,SCHALKE ROVERS,UNITED CHURCHES FOOTBALL LEAGUE


In [47]:
# Check for any null values
dcc[dcc.isnull().any(axis=1)]

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil,Area,ClubName,League


##### Update the type of pitch

In [48]:
# Check for the type of pitch available in all the other dataset
print(set(fcc["TypeOfPitch"]))
print(set(dlr["TypeOfPitch"]))

{'All weather pitches', 'Basketball Court', 'Rugby Pitches', 'GAA Pitches', 'Tennis Courts', 'Soccer pitches', 'Seven-a-side'}
{'3', '2', '9', 'Temporary pitch', '13', '12', 'Rugby', '8', 'Location of future pitchs', '1', 'GAA A', '6', '14', 'All weather', '11', '4', '5', 'A', 'B', 'Soccer', '10', 'GAA', 'GAA B', '7'}


In [49]:
## Search for GA as Gailic, FA as Football and Rugby in the Clubname and update the Pitch Type Accordingly
for i, row in dcc.iterrows():
    if any(a in dcc.iloc[i,7] for a in [" GAA"," G.A.A"]):
        dcc.iloc[i,1] = 'GAA'
    elif any(a in dcc.iloc[i,7] for a in [" A.F.C"," F.C"," FC"]):
        dcc.iloc[i,1] = 'Soccer'
    elif any(a in dcc.iloc[i,7] for a in [" RUGBY"]):
        dcc.iloc[i,1] = 'Rugby'

In [50]:
dcc.head()

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil,Area,ClubName,League
0,ALBERT COLLEGE,Soccer,Whitehall A ED,53.383004,-6.263458,Dublin City Council,NORTH WEST,DRUMCONDRA F.C (Snr),AMATEUR FOOTBALL LEAGUE
1,ALBERT COLLEGE,Not Available,Whitehall A ED,53.383004,-6.263458,Dublin City Council,NORTH WEST,GLASNEVIN AFC,AMATEUR FOOTBALL LEAGUE
2,BEECHILL,Soccer,Dublin 4,53.329195,-6.231332,Dublin City Council,SOUTH EAST,BALLSBRIDGE FC,AMATEUR FOOTBALL LEAGUE
3,BELCAMP,Not Available,Swords-Lissenhall ED,53.472847,-6.225357,Dublin City Council,NORTH CENTRAL,NEWTOWN CELTIC,AMATEUR FOOTBALL LEAGUE
4,BELCAMP,Not Available,Priorswood A ED,53.407424,-6.210297,Dublin City Council,NORTH CENTRAL,VIANNEY BOYS,AMATEUR FOOTBALL LEAGUE


##### Check the column names in all the documents

In [51]:
print(list(dlr))
print(list(dcc))
print(list(fcc))

['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude', 'CityCouncil']
['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude', 'CityCouncil', 'Area', 'ClubName', 'League']
['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude', 'CityCouncil']


#### Merge to a single dataset

In [52]:
# Copy all the datasets to new one to keep a backup
dcc_updated = dcc
dlr_updated = dlr
fcc_updated = fcc

# Keep only the relevant columns that match with other datasets for DLR and FCC
dcc_updated = dcc_updated[["Park","TypeOfPitch","Location","Latitude", "Longitude","CityCouncil"]]

# Check the column names for all the datasets
print(list(dlr_updated))
print(list(dcc_updated))
print(list(fcc_updated))

['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude', 'CityCouncil']
['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude', 'CityCouncil']
['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude', 'CityCouncil']


In [53]:
#Check the datatype of each dataset before merging 
print("******* DCC ********")
print(dcc_updated.dtypes)
print("******* DLR ********")
print(dlr_updated.dtypes)
print("******* FCC ********")
print(fcc_updated.dtypes)

******* DCC ********
Park            object
TypeOfPitch     object
Location        object
Latitude       float64
Longitude      float64
CityCouncil     object
dtype: object
******* DLR ********
Park            object
TypeOfPitch     object
Location        object
Latitude       float64
Longitude      float64
CityCouncil     object
dtype: object
******* FCC ********
Park           object
TypeOfPitch    object
Location       object
Latitude       object
Longitude      object
CityCouncil    object
dtype: object


##### As there are diffence in data types of different field make the data type same accross all datasets

In [54]:
fcc_updated['Latitude'] = fcc_updated['Latitude'].astype(np.float64)
fcc_updated['Longitude'] = fcc_updated['Longitude'].astype(np.float64)

## Recheck after modification
print("******* DCC ********")
print(dcc_updated.dtypes)
print("******* DLR ********")
print(dlr_updated.dtypes)
print("******* FCC ********")
print(fcc_updated.dtypes)

******* DCC ********
Park            object
TypeOfPitch     object
Location        object
Latitude       float64
Longitude      float64
CityCouncil     object
dtype: object
******* DLR ********
Park            object
TypeOfPitch     object
Location        object
Latitude       float64
Longitude      float64
CityCouncil     object
dtype: object
******* FCC ********
Park            object
TypeOfPitch     object
Location        object
Latitude       float64
Longitude      float64
CityCouncil     object
dtype: object


In [55]:
# Concat the three dataset to create a single dataset
final_data = pd.concat([fcc_updated,dlr_updated,dcc_updated])

# Reorder the columns in the final dataset
final_data = final_data[['Park', 'TypeOfPitch', 'Location', 'Latitude', 'Longitude', 'CityCouncil']]

# Check the final dataset
final_data.head(n=500)

,Park,TypeOfPitch,Location,Latitude,Longitude,CityCouncil
0,Balbriggan Town Park,All weather pitches,Balbriggan,53.604960,-6.182353,Fingal
1,Balheary Reservoir,All weather pitches,Swords,53.472710,-6.223015,Fingal
2,Town Park,All weather pitches,Skerries,53.577114,-6.111072,Fingal
3,St. Mologa's Park,All weather pitches,Balbriggan,53.617667,-6.189368,Fingal
4,Seagrange Park,Basketball Court,Dublin 13,53.396667,-6.135352,Fingal
5,Grace O'Malley Park,Basketball Court,Dublin 13,53.383767,-6.069940,Fingal
6,Broomfiled,GAA Pitches,Malahide,53.439158,-6.144672,Fingal
7,St. Catherine's Park,GAA Pitches,Rush,53.538198,-6.095941,Fingal
8,Malahide Demesne,GAA Pitches,Malahide,53.447818,-6.157841,Fingal
9,Swords Manor,GAA Pitches,"Brackenstown, Swords",53.459173,-6.249069,Fingal


In [56]:
### Write To a csv file 
final_data.to_csv("Combined_pitch_details.csv", encoding='utf-8', index=False)

#### Visualise the Latitude Longitude in Map View to see the data

In [57]:
##  Create separate dataset for Latitude Longitude Data
dcc_locations = dcc_updated[['Latitude', 'Longitude']]
fcc_locations = fcc_updated[['Latitude', 'Longitude']]
dlr_locations = dlr_updated[['Latitude', 'Longitude']]

In [58]:
# Convert to a list of tuples
dcc_locationlist = dcc_locations.values.tolist()
fcc_locationlist = fcc_locations.values.tolist()
dlr_locationlist = dlr_locations.values.tolist()

In [59]:
# Take those values apart from the NAN Values
dcc_locationlist= [n for n in dcc_locationlist if not np.nan in n]
fcc_locationlist= [n for n in fcc_locationlist if not np.nan in n]
dlr_locationlist= [n for n in dlr_locationlist if not np.nan in n]

In [60]:
# Plot the data in the Map 
# Green - For FCC
# Blue - For DCC
# Red - For DLR
map = folium.Map(location=[53.3498, -6.2603], zoom_start=12)
for point in range(0, len(fcc_locationlist)):
    folium.Marker(fcc_locationlist[point],
                  icon=folium.Icon(color='green')).add_to(map)

for point in range(0, len(dcc_locationlist)):
    folium.Marker(dcc_locationlist[point],
                  icon=folium.Icon(color='blue')).add_to(map)

for point in range(0, len(dlr_locationlist)):
    folium.Marker(dlr_locationlist[point],
                  icon=folium.Icon(color='red')).add_to(map)

In [61]:
# Draw the map
map